# __Compte rendu BE AML__

Authors : Tom BOISSE & Jules ESPINOUX

## __Description des données__

Notre jeu de donnée se nomme Gungor 2018 Victorian Author Attribution. Il consiste en un csv contenant deux colonnes. <br>
La première contient des extraits de 1000 mots d'auteurs anglais du 19ème siècle ayant écrit au moins 5 livres. On a exclu les 500 mots les plus courants et les moins courants, ainsi que les noms des livres et certains mots de vocabulaire trop spécifique. Après cette étape, nous avons sélectionner les 10 000 mots les plus courants et nous avons adaptés les phrases pour maintenir leur syntaxe malgré que nous ayons enlevé certains mots. Nous obtenons ainsi plus de 53 000 lignes de 1000 mots avec un vocabulaire total de 10 000 mots. Chaque phrase a aussi été normalisée et toute ponctuation a été enlevée.<br>
La deuxième colonne contient l'identifiant relatif à l'auteur ayant écrit cette phrase. Les identifiants vont de 1 à 50. <br>
Par la suite, nous noterons phrase une ligne de 1000 mots et corpus l'ensemble de toutes les phrases. 

## __Objectifs et plan d'action__

L'objectif de ce corpus de textes est d'être capable de prédire quel auteur a écrit une ligne qu'on donnerait en entrée.<br>
Nous avons remarqué que nos données d'entraînement sont standardisées donc nous n'avons pas de prétraitement à faire. Notre problème est une classification supervisée à 50 classes. Ce problème simple en apparence a ici une grande complexité lié à la forme des entrées. En effet, le texte est une entrée compliquée à gérer car il y a des notions de relations entre chaque élément du texte. Ceci s'explique grâce aux différentes régles de grammaires et de conjuguaisons. Si nous voulons obtenir le maximum de précision il faudra chercher un moyen de transformer nos textes en entrée valide pour des classifieurs tout en essayant de garder ces éléments de relations. <br>

#### Transformation des entrées

Pour cette étape cruciale pour la suite, nous avons deux pistes que nous aimerons tester. La première est le Bag of Words (BoW) et la seconde est le TF-IDF. Ces deux pistes se nomment des vectorizers car elles transforment une phrase qui est une liste de mots en un vecteur de nombre plus digeste pour des algorithmes de machine learning. <br>
L'idée de la première prendre l'entièreté du corpus, de trier chaque mot par rapport à sa fréquence d'apparition pour ensuite transformer chaque phrase en vecteur de taille 10 000 (nombre de mots différents dans notre corpus) où la donnée à l'indice i représente le nombre d'occurence dans notre phrase du ième mot le plus récurent dans notre corpus. Ainsi, si "the" est le 4ème le plus courant dans tout notre corpus et que dans notre phrase il apparait 5 fois alors après transformation de notre phrase, à l'indice 4 nous retrouverons le nombre 5.
<br>
La seconde piste est un acronyme pour Text Frequency - Inverse document frequency. Nous précalculons la IDF qui est une vecteur représentant, pour chaque mot, l'inverse de sa fréquence d'apparition dans tout le corpus. Et pour chaque phrase, quand nous voulons la transformer, nous calculons sa TF qui est la fréquence d'apparition du mot dans cette phrase. Ainsi, si nous avons "cat cat dog" et que cat apprait 5 fois dans tout le corpus et dog 9 fois, quand nous transformerons la phrase, nous obtiendrons [2/5, 2/5, 1/9]. 

#### Classification des données

Nous devons essayer de trouver une méthode qui a un bon potentiel pour apprendre à classer chaque phrase grâce à son label avec plus que 2 classes. Deux algorithmes sortent du lot pour cette tâche précise, ce sont le Naïve Bayes et le SVM. Nous allons donc dans un premier temps tester ces algorithmes avec un split de notre jeu de donnée pour estimer le potentiel de chaque méthode puis ensuite nous essayerons d'affiner les hyperparamètres pour améliorer la précision de notre modèle.

## __Transformation de nos données__

Nous allons donc appliquer ce que nous avons définit plus tôt. Nous faisons le choix d'entraîner nos deux vectorizers sur notre corpus et pas juste sur le sous ensemble d'entraînement car ceci améliore la précision des vectorizers sans impacter la transformation des données.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# loading the csv file, separate data and labels and then split
df_train = pd.read_csv("dataset/Gungor_2018_VictorianAuthorAttribution_data-train.csv", sep=",", encoding='latin-1')
X_total = df_train["text"].values
y_total = df_train["author"].values
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=42)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer # bow vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf vectorizer

In [12]:
# creating bow vectorizer + train with total data set
bow_vectorizer = CountVectorizer()
bow_vectorizer.fit(X_total)

CountVectorizer()

In [13]:
X_train_bow = bow_vectorizer.transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

In [14]:
# creating tf idf vectorizer + train with total data set
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X_total)

TfidfVectorizer()

In [15]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## __Classification des données__

Nous allons donc essayer notre 4 combinaisons. Nos deux vectorizers avec Naïve Bayes et nos deux vectorizers avec SVM.

In [16]:
from sklearn.naive_bayes import MultinomialNB # naives bayes classifier
from sklearn.svm import LinearSVC # svm classifier
from sklearn.metrics import accuracy_score # compute accuracy scores

### Naive Bayes 

In [20]:
nb_model = MultinomialNB() # naive bayes model
nb_model.fit(X_train_bow, y_train)
y_nb_bow = svm_model.predict(X_test_bow)
print(f"*-* Accuracy for BoW and Naive Bayes :  {accuracy_score(y_test, y_nb_bow):.3f}")

*-* Accuracy for BoW and Naive Bayes :  0.278


In [21]:
nb_model = MultinomialNB() # naive bayes model
nb_model.fit(X_train_tfidf, y_train)
y_nb_tfidf = svm_model.predict(X_test_tfidf)
print(f"*-* Accuracy for TFIDF and Naive Bayes :  {accuracy_score(y_test, y_nb_tfidf):.3f}")

*-* Accuracy for TFIDF and Naive Bayes :  0.979


### SVM

In [18]:
svm_model = LinearSVC(C=1) # svm model
svm_model.fit(X_train_bow, y_train)
y_svm_bow = svm_model.predict(X_test_bow)
print(f"*-* Accuracy for BoW and SVM :  {accuracy_score(y_test, y_svm_bow):.3f}")

*-* Accuracy for BoW and SVM :  0.978


In [19]:
svm_model = LinearSVC(C=1) # svm model
svm_model.fit(X_train_tfidf, y_train)
y_svm_tfidf = svm_model.predict(X_test_tfidf)
print(f"*-* Accuracy for TFIDF and SVM :  {accuracy_score(y_test, y_svm_tfidf):.3f}")

*-* Accuracy for TFIDF and SVM :  0.979


## __Affinement du modèle__

Après une première série de test, nous concluons que BoW et Naive Bayes ne sont pas performants ensemble. Nous concluons aussi que SVM et TFIDF ont toujours l'air de réussir avec des précisions très importantes. C'est pour cela que nous avons décider d'affiner cette pipeline.

In [ ]:
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np

param_grid = {'C': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]}
grid_search = GridSearchCV(LinearSVC(max_iter=10000), param_grid, cv=5, 
                           scoring='accuracy', n_jobs=-1, verbose=1)

# Fit and find best C
grid_search.fit(X_train_tfidf, y_train)

# Extract results
C_values = [params['C'] for params in grid_search.cv_results_['params']]
mean_scores = grid_search.cv_results_['mean_test_score']
std_scores = grid_search.cv_results_['std_test_score']

print(f"Optimal C: {grid_search.best_params_['C']}")
print(f"CV accuracy: {grid_search.best_score_:.4f}")
print(f"Test accuracy: {grid_search.score(X_test_tfidf, y_test):.4f}")

# Plot: C parameter vs accuracy with error bars
plt.figure(figsize=(10, 6))
plt.errorbar(C_values, mean_scores, yerr=std_scores, marker='o', capsize=5, 
             linewidth=2, markersize=8, label='CV accuracy ± std')
plt.axvline(grid_search.best_params_['C'], color='r', linestyle='--', 
            label=f"Optimal C = {grid_search.best_params_['C']}")
plt.xscale('log')
plt.xlabel('C parameter (log scale)')
plt.ylabel('Accuracy')
plt.title('K-Fold Cross-Validation: Optimal C for LinearSVC')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
